In [1]:
from common.data_pipeline.dataset import DatasetCompiler
from common.data_pipeline.MMCBNU_6000.dataset import DatasetLoader as mmcbnu
from common.data_pipeline.FV_USM.dataset import DatasetLoader as fvusm

datasets = DatasetCompiler(datasets=[mmcbnu(included_portion=0.001), fvusm(included_portion=0)])
compiled_datasets = datasets.compile_datasets()

2023-08-22 01:14:46,556 - root - INFO - Compiled train/test/validation split for MMCBNU6000
2023-08-22 01:14:46,565 - root - INFO - Compiled train/test/validation split for FV_USM
2023-08-22 01:14:46,566 - root - INFO - Concatenating train set
2023-08-22 01:14:47,673 - root - INFO - Concatenating test set
2023-08-22 01:14:47,676 - root - INFO - Concatenating validation set


In [2]:
from common.util.enums import SetType


datasets.files[SetType.VALIDATION]

[DatasetObject(path='./datasets/MMCBNU_6000/ROIs/066/L_Fore/10.bmp', name='066/L_Fore/10.bmp', label=None, mask_path=None, metadata={'finger': 'fore', 'hand': 'left', 'is_augmented': False})]